In [5]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
from __future__ import division
from __future__ import print_function

# Load scipy/numpy/matplotlib
from   scipy.linalg import expm
import matplotlib.pyplot as plt
from   pylab import *

import numpy as np

# Efficient non-conjugate updates for partially observed high-dimensonal latent Gaussian state-spaces

Consider a state-space model with states $x \sim \mathcal{N}(\mu,\Sigma)$, which is partially observed via process $y$ through an observation model that does not admit conjugate (Kalman-Bucy) measurement update.

If these non-conjugate updates depend only an a subspace $x_1 \subset x$, then approximate non-conjugate updates can be computed in this subspace and then their effects on the larger space $x$ propagated exactly in a computationally efficient manner. 

Let

$$
x_1 = \beta^\top x
$$

Then

$$
x_1 \sim \mathcal{N}( \beta^\top \mu, \beta^\top \Sigma \beta)
$$

Define $x_2$ in terms of the subspace of $x$ that is orthogonal to $x_1$

$$
\Pr(x) = \Pr(x_1,x_2) = \Pr(x_2|x_1)\Pr(x_1)
$$

The restriction that observations $y$ depend only on the space $x_1$ implies that $y$ is independent of $x_2$, conditioned on $x_1$:

$$
\Pr(y|x) = \Pr(y|x_1,x_2) = \Pr(y|x_1)
$$

Consider the full Bayesian measurement update

$$
\Pr(x|y) = \Pr(y|x) \frac {\Pr(x)} {\Pr(y)}
$$


This can be re-phrased in terms of conditional probabilities of $x_2$ given $x_1$, and using the fact that $y$ is independent of $x_2$, conditioned on $x_1$:

$$
\Pr(x|y) = \Pr(x_2|x_1) \Pr(x_1|y) = \Pr(x_2|x_1) \left( \Pr(y|x_1) \frac {\Pr(x_1)} {\Pr(y)} \right)
$$

If we can compute $\Pr(x_1|y)$, then we can compute $\Pr(x|y)$. We may focus then on updating the subspace $x_1$ directly coupled to observations $y$

$$
\Pr(x_1|y) = \Pr(y|x_1) \frac {\Pr(x_1)} {\Pr(y)} 
$$

Assume we have an approximate Gaussian posterior for $Q(x_1) \approx \Pr(x_1|y)$, computed by Laplace approximation, variational Bayes, moment-matching, or other means. 

$$
\hat x_1 \sim \mathcal{N}( \hat\mu_1, \hat\Sigma_1 )
$$

Since $Q(x_1)$ is gaussian, it is straightforward to compute the full joint approximation update as

$$
Q(x) = \Pr(x_2|x_1) Q(x_1)
$$


However, naive multiplication of the Gaussian distributions is not computationally efficient, as it will solving linear systems of the full dimension of $x$. There are many ways to derive the more efficient update, which involves solving linear systems only as large as $x_1$. 

# The Kalman filter update

Here I present a conceptual approach that makes an explcit connection to the Kalman filter update. The Kalman filter update involves solving a linear system of the same dimensionality as the *measurements*, which is typically much smaller than the full latent state space. This leads to substantial savings.

To review, the Kalman filter update assumes a linear observation model with additive Gaussian noise

$$
z \sim \mathcal{N}(\beta^\top x,R)
$$

Given an estimate of the latent state $x \sim \mathcal{N}(\mu,\Sigma)$, the Kalman update computes a posterior $\hat x \sim \mathcal{N}(\hat \mu,\hat \Sigma)$ as follows

\begin{align*}
       \tilde z &= z - \beta^\top \mu
    \\ S &= R + \beta^\top \Sigma \beta
    \\ K &= \Sigma \beta S^{-1}
    \\ \hat\mu &= \mu + K \tilde z
    \\ J &= I - K \beta^\top
    \\ \hat\Sigma &= J \Sigma J^\top + K R K^\top
\end{align*}

Note that the only matrix inversion, $S^{-1}$, has the dimensionality of $z$ and not $x$.

# A surrogate gaussian update

We can interpret the Gaussian approximation for $x_1$ given $y$ as providing a surrogate measurement of the projection $\beta^\top x$. 

This is useful not only as a short-hand way of understanding how to do an efficent non-conjugate update, but also because the surrogate updates may be used to accelerate optimization of the parameters of the model. 

Having estimated

$$
Q(x_1) = \mathcal{N}( \hat\mu_1, \hat\Sigma_1 )
$$


We may solve for a surrogate Gaussian approximation of the *likelihood* $Q(y|x_1)$

$$
Q(y|x_1) = Q(x_1) \frac {\Pr(y)} {\Pr(x_1)} 
$$

For now let us present a solution for $Q(y|x_1)$ in terms of the naive formula for dividing two Gaussian distributions: 

\begin{align*}
    Q(y|x_1)  &= \mathcal{N} (\mu_r, \Sigma_r )
    \\ \Sigma_r &= \left(\hat \Sigma_1^{-1} - \Sigma_1^{-1} \right)^{-1}
    \\ \mu_r &= \Sigma_r \left( \hat \Sigma_1^{-1} \hat \mu_1 - \Sigma_1 ^{-1} \mu_1 \right)
\end{align*}


We may equate this posterior to the observation model

$$
z =  \mu_r
$$

$$
R = \Sigma_r
$$



In which case the full model may be updated as

\begin{align*}
       \tilde z &= \mu_r - \beta^\top \mu
    \\ S &=  \Sigma_r + \beta^\top \Sigma \beta
    \\ K &= \Sigma \beta S^{-1}
    \\ \hat\mu &= \mu + K \tilde z
    \\ J &= I - K \beta^\top
    \\ \hat\Sigma &= J \Sigma J^\top + K R K^\top
\end{align*}

So far, this connection to Kalman filtering is somewhat un-necessary, as the full update may be more directly described in terms of a conditional Gaussian, without the step of computing the surrogate likelihood. 

The Kalman update conncetion, however, is useful also for one approach to optimization of the parameters of models that include non-conjugate updates. If computing the non-conjugate update is costly, we may convert this update to a conjugate update using the surrogate Gaussian likelihoods. Parameters can be optimized then against this model, the surrograte likelihoods re-computed, and the overall model optimized iteratively via coordinate ascent. 


# Surrogate likelihoods for optimization

I think we should be able to recover the likelihood by negating the covariance in the Kalman update

\begin{align*}
       \tilde z &= z - \beta^\top \mu
    \\ S &= R - \beta^\top \Sigma \beta
    \\ K &= - \Sigma \beta S^{-1}
    \\ \hat\mu &= \mu + K \tilde z
    \\ J &= I - K \beta^\top
    \\ \hat\Sigma &= -J \Sigma J^\top + K R K^\top
\end{align*}

This would be done in the $x_1$ subspace

\begin{align*}
       \tilde z &= \hat\mu_1 - \beta^\top \mu
    \\ S &= \hat\Sigma_1 - \beta^\top \Sigma \beta
    \\ K &= \Sigma \beta S^{-1}
    \\ \hat\mu &= \mu - K \tilde z
    \\ J &= I + K \beta^\top
    \\ \hat\Sigma &= K \hat\Sigma_1 K^\top - J \Sigma J^\top
\end{align*}

\begin{align*}
       \tilde z &= z - \beta^\top \mu
    \\ S &= R + \beta^\top \Sigma \beta
    \\ K &= \Sigma \beta S^{-1}
    \\ \hat\mu &= \mu + K \tilde z
    \\ J &= I - K \beta^\top
    \\ \hat\Sigma &= J \Sigma J^\top + K R K^\top
\end{align*}

This is not really all that much better

In [23]:
N = 10
I = np.eye(N)

# Invent a prior
prm = randn(N)
prq = randn(N,N)
prv = prq.dot(prq.T)

# Invent a likelihood / measurement
lkm = randn(N)
lkq = randn(N,N)
lkv = prq.dot(prq.T)

def kalman_product(prm,prv,lkm,lkv):
    # Kalman update to get postrior
    z = lkm - prm
    S = lkv + prv
    K = prv.dot(pinv(S))
    psmk = prm + K.dot(z)
    J = I - K
    psvk = J.dot(prv).dot(J.T) + K.dot(lkv).dot(K.T)
    return psmk,psvk

def bayesian_product(prm,prv,lkm,lkv):
    # Bayesian update to get posterior
    psvb = pinv(pinv(prv)+pinv(lkv))
    psmb = psvb.dot(pinv(prv).dot(prm) + pinv(lkv).dot(lkm))
    return psmb,psvb

# Verif they are the same
psmk,psvk = kalman_product(prm,prv,lkm,lkv)
psmb,psvb = bayesian_product(prm,prv,lkm,lkv)
print(sum(abs(psvk-psvb)),sum(abs(psmk-psmb)))

# Verify likelihood recovered by performing update with negative prior covariance
rrmk,rrvk = kalman_product(prm,-prv,psmk,psvk)
print(sum(abs(rrvk-lkv)),sum(abs(rrmk-lkm)))
rrmb,rrvb = bayesian_product(prm,-prv,psmb,psvb)
print(sum(abs(rrvb-lkv)),sum(abs(rrmb-lkm)))

1.18714239827e-11 1.56268054052e-12
1.58081880919e-13 4.31184254968e-12
4.59201357494e-11 4.50718629086e-12
